In [51]:
import numpy as np
import pandas as pd

# Define the Pauli matrices without Y
I = np.array([[1, 0], [0, 1]])
X = np.array([[0, 1], [1, 0]])
Y = np.array([[0, -1j], [1j, 0]])
Z = np.array([[1, 0], [0, -1]])


k_1p = 5
k_1m = 3
k_2p = 4
k_2m = 2
k_3p = 3
k_3m = 1

# Define the given matrix A
# A = np.array([
#     [-5, 3, 0, 0],
#     [5, -7, 2, 0],
#     [0, 4, -5, 1],
#     [0, 0, 3, -1]
# ])

A = np.array([
    [-k_1p, k_1m, 0, 0],
    [k_1p, -(k_1m + k_2p), k_2m, 0],
    [0, k_2p, -(k_2m + k_3p), k_3m],
    [0, 0, k_3p, -k_3m]])




# Define the given matrix A
A = np.array([
    [-5, 3, 0, 0],
    [5, -7, 2, 0],
    [0, 4, -5, 1],
    [0, 0, 3, -1]
])

# List of Pauli matrices
paulis = [I, X, Y, Z]
pauli_labels = ['I', 'X', 'Y', 'Z']

# Function to calculate the coefficients
def calculate_coefficients(A):
    coefficients = {}
    for i, sigma_i in enumerate(paulis):
        for j, sigma_j in enumerate(paulis):
            pauli_string = f"{pauli_labels[i]} \u2297 {pauli_labels[j]}"
            tensor_product = np.kron(sigma_i, sigma_j)
            coefficient = 0.25 * np.trace(A @ tensor_product)
            coefficients[pauli_string] = coefficient
    return coefficients

# Calculate the coefficients
coefficients = calculate_coefficients(A)

print(coefficients)




{'I ⊗ I': -4.5, 'I ⊗ X': 3.0, 'I ⊗ Y': -1j, 'I ⊗ Z': -0.5, 'X ⊗ I': 0.0, 'X ⊗ X': 1.5, 'X ⊗ Y': 0.5j, 'X ⊗ Z': 0.0, 'Y ⊗ I': 0j, 'Y ⊗ X': -0.5j, 'Y ⊗ Y': (1.5+0j), 'Y ⊗ Z': 0j, 'Z ⊗ I': -1.5, 'Z ⊗ X': 1.0, 'Z ⊗ Y': 0j, 'Z ⊗ Z': 1.5}


In [42]:
#analyical eigenvalues

eigenvalues = np.linalg.eigvals(A)

print(eigenvalues)

[-1.09424629e+01 -5.44685237e+00 -1.61068475e+00 -1.23787751e-16]


In [43]:
import qiskit
import numpy as np
from qiskit.circuit import QuantumCircuit, ParameterVector

from qiskit.quantum_info import SparsePauliOp
from qiskit_algorithms.optimizers import SPSA
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import TwoLocal
from qiskit_algorithms.utils import algorithm_globals
from qiskit.primitives import Sampler
from qiskit.primitives import Estimator
from qiskit_ibm_runtime import QiskitRuntimeService, Session
import numpy as np
from qiskit_algorithms import VQE
import pylab
from qiskit import QuantumCircuit
from qiskit import transpile
from qiskit.providers.basic_provider import BasicSimulator
from qiskit.quantum_info import Statevector
import pandas as pd

from qiskit_algorithms import NumPyMinimumEigensolver


In [49]:
#set the coefficents to a coeff list

coeff = list(coefficients.values())

Pauli_strings = ['II', 'IX', 'IZ', 'XI', 'XX', 'XZ', 'ZI', 'ZX', 'ZZ']

SPSA_obj = SPSA(maxiter=100)

Hamiltonian = SparsePauliOp(Pauli_strings, coeff)

counts = []
values = []

def store_intermediate_result(eval_count, parameters, mean, std):
    counts.append(eval_count)
    values.append(mean)


def run_vqe(Hamiltonian, ansatz, optimizer, backend):
    with Session(backend=backend) as session:
        estimator = Estimator()
        vqe = VQE(
            estimator=estimator,
            ansatz=ansatz,
            optimizer=SPSA_obj,
            callback=store_intermediate_result,
    )
    result = vqe.compute_minimum_eigenvalue(Hamiltonian)

    return result



[[-5. +0.j  4. +0.j  0. +0.j  1.5+0.j]
 [ 4. +0.j -7. +0.j  1.5+0.j  0. +0.j]
 [ 0. +0.j  1.5+0.j -5. +0.j  2. +0.j]
 [ 1.5+0.j  0. +0.j  2. +0.j -1. +0.j]]


In [45]:
from qiskit.circuit.library import EfficientSU2

from qiskit_ibm_runtime.fake_provider import FakeManilaV2

num_qubits = 2

ansatz = EfficientSU2(num_qubits, reps=1)

x0 = 2* np.pi * np.random.rand(ansatz.num_parameters)

QiskitRuntimeService.save_account(channel="ibm_quantum",token="2ec3d4b6d5627dd41d59512bd3f409639bc03dc9ec4dea96fe4067cfa8068a4598d9425f65f83bcde69f36ec0163d27636038af17292aea7db49ff6691b7be1a",overwrite=True)
service = QiskitRuntimeService()
backend = FakeManilaV2()

result = run_vqe(Hamiltonian, ansatz, SPSA_obj, backend)

#analytical solution
    
numpy_solver = NumPyMinimumEigensolver()
result_exact = numpy_solver.compute_minimum_eigenvalue(Hamiltonian)
ref_value = result_exact.eigenvalue.real

print("VQE Result:", result.eigenvalue.real)
print("Exact Result:", ref_value)


VQE Result: -10.57822590607047
Exact Result: -10.578915463534539


In [46]:
#VQD for the second and third lowest eigenvalues
from qiskit_algorithms import VQD, VQDResult

eval_counts = []
values = []
current_steps = []

def store_intermediate_result(eval_count, parameters, value, meta, step):
    eval_counts.append(eval_count)
    values.append(value)
    current_steps.append(step)

def run_vqd(Hamiltonian, ansatz, optimizer, backend, fidelity, k):
    with Session(backend=backend) as session:
        estimator = Estimator()
        vqd = VQD(
            estimator=estimator,
            ansatz=ansatz,
            optimizer=optimizer,
            callback=store_intermediate_result,
            k=k,
            fidelity=fidelity
        )
    result = vqd.compute_eigenvalues(Hamiltonian)

    return result

In [47]:
from qiskit_algorithms.state_fidelities import ComputeUncompute
from qiskit.primitives import Sampler

num_qubits = 2

SPSA_obj = SPSA(maxiter=100)

ansatz = EfficientSU2(num_qubits, reps=1)

x0 = 2* np.pi * np.random.rand(ansatz.num_parameters)

QiskitRuntimeService.save_account(channel="ibm_quantum",token="2ec3d4b6d5627dd41d59512bd3f409639bc03dc9ec4dea96fe4067cfa8068a4598d9425f65f83bcde69f36ec0163d27636038af17292aea7db49ff6691b7be1a",overwrite=True)
service = QiskitRuntimeService()
backend = FakeManilaV2()

sampler = Sampler()
fidelity = ComputeUncompute(sampler)


#have to declare a fidelity object

result = run_vqd(Hamiltonian, ansatz, SPSA_obj, backend, fidelity, 3)

print("VQD Result:", result.eigenvalues)

VQD Result: [-10.52324729+0.j  -2.45118486+0.j  -1.22648645+0.j]
